In [62]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torchvision import models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

In [63]:
transform = transforms.Compose([
    transforms.Resize(256),             
    transforms.CenterCrop(224),        
    transforms.ToTensor(),             
    transforms.Normalize(              
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

In [64]:
class CSVImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = 'data/images/' + self.df.iloc[idx]
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
            
        return image
    def to_tensor(self):
        tensor = []
        for i in range(len(self)):
            img_tensor = self[i]
            tensor.append(img_tensor)
        return torch.stack(tensor)

In [65]:
data = pd.read_csv('data/train.csv')
X = data['image']
y = data['class']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1000)

In [67]:
X_train = CSVImageDataset(X_train, transform=transform)
X_train = X_train.to_tensor()
X_test = CSVImageDataset(X_test, transform=transform)
X_test = X_test.to_tensor()
y_train = torch.from_numpy(np.array(y_train)).long()
y_test = torch.from_numpy(np.array(y_test)).long()
y_test

tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [68]:
weights = models.AlexNet_Weights.DEFAULT
rostelecomnet = models.alexnet(weights=weights)
print(rostelecomnet)
for param in rostelecomnet.parameters():
    param.requires_grad = False
rostelecomnet.classifier[6] = torch.nn.Linear(rostelecomnet.classifier[6].in_features, 2)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [69]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(rostelecomnet.parameters(), lr=1.0e-2)

In [70]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
rostelecomnet = rostelecomnet.to(device)
X_test = X_test.to(device)

In [71]:
batch_size = 253
losses = []
tests = []

for epoch in range(5000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        batch_indexes = order[start_index:start_index+batch_size]
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        preds = rostelecomnet.forward(x_batch)
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        optimizer.step()
    if epoch % 10 == 0:
        test_preds = rostelecomnet.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print(f1_score(test_preds, y_test)) 
        if f1_score(test_preds, y_test) >= 0.995:
            break
        

0.8961038961038961
0.9787234042553191
0.9855072463768116
0.9784172661870504
0.9928057553956835
1.0


In [75]:
def predict(net, x):
    y_pred = net.forward(x)
    y_pred = y_pred.argmax(dim=1)
    return y_pred

In [76]:
test_data = pd.read_csv('data/test.csv')
images = test_data['image']
test = CSVImageDataset(images, transform=transform)
test = test.to_tensor()

In [77]:
y_preds = predict(rostelecomnet, test)

test_data['class'] = y_preds.detach().numpy()
test_data.to_csv('data/submit.csv')
test_data

,image,class
0,pnlioc.jpg,1
1,hivmnw.jpg,1
2,fsftlx.jpg,1
3,uygvah.jpg,0
4,kmcfhr.jpg,1
...,...,...
156,dovvvc.jpg,1
157,axrglz.jpg,1
158,xyxrkm.jpg,0
159,xosnep.jpg,1
